# Problem Representation Design Pattern

This chapter looks at the different types of machine learning problems and analyses how the model architectures vary depending on the problem.

The input and output types are tow key factors impacting the model architecture. For example the output required from a model could impact if we choose a regression or a classification model. Special nerual network layers exist for specific types of input data: convolutional layers for images, speech text and other data with spatiotemporal correlation, recurrent networks for sequential data. Special classes of solutions exist for commonly occuring problems like recommendations (matrix factorization), or time-series (ARIMA). A group of simpler model stogher with common isioms can be used to solve more compex problems e.g. text generation often involves a classification model whose outputs are postprocessed using a beam search algorithm.

## Design Pattern 5: Reframing

This pattern refers to chaning the ouput of ML problem. For example, we could take something that is intuitively a regression problem and instead pose it as a classification prblem.

### Problem

The first step of building any ML solution is framing the problem. Is this a supervised learning problem? Or unsupervised? What are the features? If it is a supervised problem what are the labels? What amount of error is acceptable? Of course, the answers to these questions must be considered in context with the training, the task at hand, and the metrics for success.

For example, if we wanted to predict the amount of rainfall in a given area we could make this a regression problem. We could also treat this as a time series model. There are lost of adjustments we can make to improve our model. Is regression the onle wat we can pose this task? Perhaps we can re-frame our machine learning objective in a way that improves our task performance.